In [ ]:
!pip install -q transformers librosa wandb face-alignment dlib yacs
!git clone https://github.com/OpenTalker/SadTalker.git 2>/dev/null || true

# Download pretrained checkpoints
!mkdir -p SadTalker/checkpoints
!wget -q -nc "https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2/mapping_00109-model.pth.tar" -O SadTalker/checkpoints/mapping.pth.tar
!wget -q -nc "https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2/SadTalker_V0.0.2_256.safetensors" -O SadTalker/checkpoints/sadtalker_256.safetensors
!wget -q -nc "https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2/BFM_Fitting.zip" -O /tmp/BFM_Fitting.zip && unzip -qo /tmp/BFM_Fitting.zip -d SadTalker/checkpoints/ 2>/dev/null || true

echo "If downloads fail, see: https://github.com/OpenTalker/SadTalker#-2-download-trained-models"

In [ ]:
import sys
sys.path.insert(0, "/content")
sys.path.insert(0, "/content/SadTalker")
sys.path.insert(0, "/content/SadTalker/src")

import gc
import json
import warnings
from pathlib import Path

import cv2
import librosa
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import wandb
from torch.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from emotion_utils import (
    CrossModalEmotionLoss,
    DifferentiableVideoPreprocess,
    EmotionAgreementMetric,
    load_frozen_audio_encoder,
    load_frozen_video_encoder,
    extract_audio_embedding,
    extract_video_embedding,
)

warnings.filterwarnings("ignore")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
METADATA = "/content/processed_data/metadata.json"
BEST_AUDIO_PATH = "/content/trained_encoders_v2/hubert-er-lr5e5"
BEST_VIDEO_PATH = "/content/trained_encoders_v2/tsf-lr3e5-16f"
OUT_DIR = Path("/content/sadtalker_finetuned")
OUT_DIR.mkdir(parents=True, exist_ok=True)

EXCLUDE = {0, 7}
REMAP = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}

print(f"Device: {DEVICE}")

In [ ]:
"""
SadTalker pipeline overview:
  Audio -> AudioEncoder -> ExpNet -> expression coefficients (3DMM)
  Expression coefficients + source face -> FaceRenderer -> generated video

We fine-tune the ExpNet (expression mapper) so that generated expressions
carry the same emotion as detected in the audio by our frozen encoders.

Gradient path: emotion_loss -> generated_frames -> renderer -> expression_coeffs -> ExpNet
"""

from src.audio2pose_models.audio2pose import Audio2Pose
from src.audio2exp_models.audio2exp import Audio2Exp
from src.audio2exp_models.networks import SimpleWrapperV2
from src.facerender.animate import AnimateFromCoeff
from src.generate_batch import get_data
from src.generate_facerender_batch import get_facerender_data
from src.utils.preprocess import CropAndExtract

SADTALKER_CKPT = Path("/content/SadTalker/checkpoints")

print("SadTalker modules loaded.")

In [ ]:
class ExpNetWrapper(nn.Module):
    """Wraps SadTalker's Audio2Exp for fine-tuning.
    Only expression mapper is trainable; audio backbone stays frozen."""

    def __init__(self, audio2exp_model):
        super().__init__()
        self.model = audio2exp_model

    def freeze_audio_backbone(self):
        for name, p in self.model.named_parameters():
            if "netG" not in name:
                p.requires_grad = False

    def forward(self, batch):
        return self.model(batch)


class EmotionAwareFaceRenderer(nn.Module):
    """Wraps SadTalker face renderer so output is differentiable for emotion loss."""

    def __init__(self, renderer):
        super().__init__()
        self.renderer = renderer

    @torch.no_grad()
    def render(self, source_img, coeffs_dict):
        """Render face given 3DMM coefficients. Frozen -- no gradients."""
        return self.renderer.generate(source_img, coeffs_dict)


def load_sadtalker_components(ckpt_dir, device):
    ckpt_dir = Path(ckpt_dir)

    from src.utils.init_path import init_path
    sadtalker_paths = init_path(
        str(ckpt_dir), "/content/SadTalker/src/config",
        "256", False, "crop"
    )

    preprocess_model = CropAndExtract(sadtalker_paths, device)

    from src.test_audio2coeff import Audio2Coeff
    audio2coeff = Audio2Coeff(sadtalker_paths, device)

    from src.facerender.animate import AnimateFromCoeff
    animate = AnimateFromCoeff(sadtalker_paths, device)

    return preprocess_model, audio2coeff, animate


try:
    preprocess_model, audio2coeff, animate = load_sadtalker_components(SADTALKER_CKPT, DEVICE)
    exp_net = ExpNetWrapper(audio2coeff.audio2exp)
    exp_net.freeze_audio_backbone()
    trainable = sum(p.numel() for p in exp_net.parameters() if p.requires_grad)
    total = sum(p.numel() for p in exp_net.parameters())
    print(f"ExpNet: {trainable/1e6:.1f}M trainable / {total/1e6:.1f}M total")
except Exception as e:
    print(f"SadTalker loading issue (expected on first run): {e}")
    print("Adjust checkpoint paths in cell 0 and re-run.")

In [ ]:
audio_enc, audio_proc = load_frozen_audio_encoder(BEST_AUDIO_PATH, DEVICE)
video_enc = load_frozen_video_encoder(BEST_VIDEO_PATH, DEVICE)
video_preprocess = DifferentiableVideoPreprocess(224).to(DEVICE)
print("Frozen emotion encoders loaded.")

In [ ]:
SR = 16000
IMG_SIZE = 256

class SadTalkerDataset(Dataset):
    def __init__(self, metadata_path, split, n_frames=8):
        with open(metadata_path) as f:
            data = json.load(f)
        self.samples = [s for s in data
                        if s["split"] == split and s["emotion_idx"] not in EXCLUDE]
        self.n_frames = n_frames

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s = self.samples[idx]

        wav, sr = torchaudio.load(s["audio_path"])
        audio_1d = wav.squeeze(0)

        frames = np.load(s["frames_path"]).astype(np.float32) / 255.0
        n = frames.shape[0]

        source_idx = 0
        source = torch.from_numpy(frames[source_idx]).permute(2, 0, 1)
        if source.shape[1] != IMG_SIZE or source.shape[2] != IMG_SIZE:
            source = F.interpolate(source.unsqueeze(0), size=(IMG_SIZE, IMG_SIZE),
                                   mode="bilinear", align_corners=False).squeeze(0)

        indices = np.linspace(0, n - 1, self.n_frames).astype(int)
        gt_frames = torch.from_numpy(frames[indices]).permute(0, 3, 1, 2)
        if gt_frames.shape[2] != IMG_SIZE or gt_frames.shape[3] != IMG_SIZE:
            gt_frames = F.interpolate(gt_frames, size=(IMG_SIZE, IMG_SIZE),
                                      mode="bilinear", align_corners=False)

        return {
            "audio": audio_1d,
            "audio_path": s["audio_path"],
            "source": source,
            "gt_frames": gt_frames,
            "emotion": REMAP[s["emotion_idx"]],
        }


def collate_sadtalker(batch):
    return {
        "audio": [b["audio"] for b in batch],
        "audio_path": [b["audio_path"] for b in batch],
        "source": torch.stack([b["source"] for b in batch]),
        "gt_frames": torch.stack([b["gt_frames"] for b in batch]),
        "emotion": torch.tensor([b["emotion"] for b in batch]),
    }


train_ds = SadTalkerDataset(METADATA, "train", n_frames=8)
val_ds = SadTalkerDataset(METADATA, "val", n_frames=8)
print(f"Train: {len(train_ds)}, Val: {len(val_ds)}")

In [ ]:
"""
SadTalker fine-tuning strategy:
  1. Freeze: audio backbone, pose network, face renderer
  2. Train: expression network (ExpNet / netG)
  3. Loss: expression_coeff_loss + lambda_emo * cross_modal_emotion_loss

Two loss paths:
  (a) Coefficient loss -- L1 between predicted and GT 3DMM expression coefficients
      (requires extracting GT coefficients from real faces)
  (b) Emotion loss -- cosine distance between audio and video emotion embeddings
      (requires rendering -> differentiable preprocess -> frozen emotion encoder)

For efficiency, path (b) is computed every N steps (rendering is expensive).
"""

def coeff_loss_fn(pred_coeffs, gt_coeffs):
    return F.l1_loss(pred_coeffs, gt_coeffs)


@torch.no_grad()
def extract_3dmm_coeffs(preprocess_model, source_img, device):
    """Extract 3DMM coefficients from a face image using SadTalker's preprocessor."""
    if isinstance(source_img, torch.Tensor):
        img = (source_img.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return preprocess_model.generate(img, device=device)

In [ ]:
wandb.login()

CONFIGS = [
    {"name": "sadtalker-baseline", "lambda_emo": 0.0},
    {"name": "sadtalker-emo-001",  "lambda_emo": 0.01},
    {"name": "sadtalker-emo-005",  "lambda_emo": 0.05},
    {"name": "sadtalker-emo-01",   "lambda_emo": 0.1},
]

LR = 5e-5
EPOCHS = 15
BATCH_SIZE = 2
PATIENCE = 5
EMO_EVAL_EVERY = 5

In [ ]:
def generate_with_sadtalker(audio2coeff, animate, audio_path, source_img, device):
    """Full SadTalker inference for a single sample. Returns generated frames (T, C, H, W)."""
    from src.generate_batch import get_data as get_batch_data

    batch = get_batch_data(audio2coeff, audio_path, device)
    coeff_dict = audio2coeff.generate(batch, save_path=None, return_coeffs=True)
    rendered = animate.generate(source_img, coeff_dict)
    return rendered


def train_one_epoch(exp_net, train_loader, optimizer, scaler, emotion_loss_fn,
                    lambda_emo, audio2coeff, animate, step_counter):
    exp_net.model.netG.train()
    total_loss, total_emo = 0.0, 0.0
    n = 0

    for batch in tqdm(train_loader, leave=False):
        source = batch["source"].to(DEVICE)
        B = source.shape[0]

        optimizer.zero_grad(set_to_none=True)

        emo_loss = torch.tensor(0.0, device=DEVICE)
        with autocast("cuda", enabled=DEVICE == "cuda"):
            if lambda_emo > 0 and step_counter[0] % EMO_EVAL_EVERY == 0:
                gen_frames_list = []
                for i in range(B):
                    try:
                        frames = generate_with_sadtalker(
                            audio2coeff, animate,
                            batch["audio_path"][i], source[i], DEVICE)
                        if isinstance(frames, torch.Tensor) and frames.dim() == 4:
                            gen_frames_list.append(frames)
                    except Exception:
                        continue

                if gen_frames_list:
                    n_frames = min(f.shape[0] for f in gen_frames_list)
                    gen_video = torch.stack([f[:n_frames] for f in gen_frames_list])
                    gen_video = gen_video.float() / 255.0 if gen_video.max() > 1 else gen_video.float()

                    audio_emb = extract_audio_embedding(
                        audio_enc, audio_proc, batch["audio"], device=DEVICE)
                    video_emb = extract_video_embedding(
                        video_enc, video_preprocess, gen_video, device=DEVICE)
                    emo_loss = emotion_loss_fn(audio_emb.detach(), video_emb)

            loss = lambda_emo * emo_loss if lambda_emo > 0 else torch.tensor(0.0, device=DEVICE)

        if loss.requires_grad:
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(
                [p for p in exp_net.parameters() if p.requires_grad], 1.0)
            scaler.step(optimizer)
            scaler.update()

        total_loss += loss.item()
        total_emo += emo_loss.item()
        step_counter[0] += 1
        n += 1

    return {"total": total_loss / max(n, 1), "emotion": total_emo / max(n, 1)}


@torch.no_grad()
def evaluate(exp_net, val_loader, emotion_loss_fn, lambda_emo, audio2coeff, animate):
    exp_net.model.netG.eval()
    metric = EmotionAgreementMetric()
    total_emo = 0.0
    n = 0

    for batch in tqdm(val_loader, leave=False):
        source = batch["source"].to(DEVICE)
        B = source.shape[0]

        gen_frames_list = []
        for i in range(B):
            try:
                frames = generate_with_sadtalker(
                    audio2coeff, animate,
                    batch["audio_path"][i], source[i], DEVICE)
                if isinstance(frames, torch.Tensor) and frames.dim() == 4:
                    gen_frames_list.append(frames)
            except Exception:
                continue

        if not gen_frames_list:
            continue

        n_frames = min(f.shape[0] for f in gen_frames_list)
        gen_video = torch.stack([f[:n_frames] for f in gen_frames_list])
        gen_video = gen_video.float() / 255.0 if gen_video.max() > 1 else gen_video.float()

        audio_emb = extract_audio_embedding(
            audio_enc, audio_proc, batch["audio"], device=DEVICE)
        video_emb = extract_video_embedding(
            video_enc, video_preprocess, gen_video, device=DEVICE)

        emo_loss = emotion_loss_fn(audio_emb, video_emb)
        total_emo += emo_loss.item()
        metric.update(audio_emb, video_emb)
        n += 1

    result = {"emotion": total_emo / max(n, 1)}
    result.update(metric.compute())
    return result

In [ ]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=0, collate_fn=collate_sadtalker)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=0, collate_fn=collate_sadtalker)

all_results = []

for cfg in CONFIGS:
    name = cfg["name"]
    lambda_emo = cfg["lambda_emo"]
    print(f"\n{'='*60}\n{name} (lambda_emo={lambda_emo})\n{'='*60}")

    wandb.init(project="uncanny-valley-sadtalker", name=name,
               config={**cfg, "lr": LR, "epochs": EPOCHS}, reinit=True)

    preprocess_model, audio2coeff, animate = load_sadtalker_components(SADTALKER_CKPT, DEVICE)
    exp_net = ExpNetWrapper(audio2coeff.audio2exp)
    exp_net.freeze_audio_backbone()

    trainable_params = [p for p in exp_net.parameters() if p.requires_grad]
    optimizer = torch.optim.AdamW(trainable_params, lr=LR)
    scaler = GradScaler(enabled=DEVICE == "cuda")
    emotion_loss_fn = CrossModalEmotionLoss(weight=1.0)

    best_val, patience_cnt = float("inf"), 0
    save_path = OUT_DIR / name
    step_counter = [0]

    for epoch in range(EPOCHS):
        t = train_one_epoch(exp_net, train_loader, optimizer, scaler,
                            emotion_loss_fn, lambda_emo, audio2coeff, animate, step_counter)
        v = evaluate(exp_net, val_loader, emotion_loss_fn, lambda_emo, audio2coeff, animate)

        wandb.log({
            "epoch": epoch + 1,
            "train/total": t["total"], "train/emotion": t["emotion"],
            "val/emotion": v["emotion"],
            **{f"val/{k}": v[k] for k in ["avg_cosine_sim", "agreement_rate"] if k in v},
        })

        print(f"  [{epoch+1:2d}/{EPOCHS}] "
              f"t_loss={t['total']:.4f} v_emo={v['emotion']:.4f}"
              + (f" cos_sim={v.get('avg_cosine_sim', 0):.3f}" if lambda_emo > 0 else ""))

        val_metric = v["emotion"] if lambda_emo > 0 else v.get("avg_cosine_sim", float("inf"))
        if val_metric < best_val:
            best_val = val_metric
            save_path.mkdir(parents=True, exist_ok=True)
            torch.save(
                {k: v for k, v in exp_net.model.netG.state_dict().items()},
                save_path / "expnet.pth")
            patience_cnt = 0
        else:
            patience_cnt += 1
            if patience_cnt >= PATIENCE:
                print(f"  Early stopping at epoch {epoch+1}")
                break

    wandb.finish()
    del preprocess_model, audio2coeff, animate, exp_net, optimizer, scaler
    torch.cuda.empty_cache()
    gc.collect()
    all_results.append({"name": name, "lambda_emo": lambda_emo, "best_val": best_val})
    print(f"  Best val metric: {best_val:.4f} -> {save_path}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(all_results).sort_values("best_val")
print(df.to_string(index=False))

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(df["name"], df["best_val"], color="coral")
ax.set_ylabel("Best Val Emotion Loss")
ax.set_title("SadTalker Fine-tuning: λ_emo Ablation")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
best_name = df.iloc[0]["name"]
print(f"Best SadTalker variant: {best_name}")

preprocess_model, audio2coeff, animate = load_sadtalker_components(SADTALKER_CKPT, DEVICE)
exp_net = ExpNetWrapper(audio2coeff.audio2exp)

ckpt = torch.load(OUT_DIR / best_name / "expnet.pth", map_location=DEVICE, weights_only=True)
exp_net.model.netG.load_state_dict(ckpt)
exp_net.eval()

metric = EmotionAgreementMetric()

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating best"):
        source = batch["source"].to(DEVICE)
        B = source.shape[0]

        gen_frames_list = []
        for i in range(B):
            try:
                frames = generate_with_sadtalker(
                    audio2coeff, animate,
                    batch["audio_path"][i], source[i], DEVICE)
                if isinstance(frames, torch.Tensor) and frames.dim() == 4:
                    gen_frames_list.append(frames)
            except Exception:
                continue

        if not gen_frames_list:
            continue

        n_frames = min(f.shape[0] for f in gen_frames_list)
        gen_video = torch.stack([f[:n_frames] for f in gen_frames_list])
        gen_video = gen_video.float() / 255.0 if gen_video.max() > 1 else gen_video.float()

        audio_emb = extract_audio_embedding(
            audio_enc, audio_proc, batch["audio"], device=DEVICE)
        video_emb = extract_video_embedding(
            video_enc, video_preprocess, gen_video, device=DEVICE)
        metric.update(audio_emb, video_emb)

agreement = metric.compute()
print(f"\nBest model evaluation:")
print(f"  Avg cosine sim:   {agreement['avg_cosine_sim']:.4f}")
print(f"  Agreement rate:   {agreement['agreement_rate']:.4f}")
print(f"  Std cosine sim:   {agreement['std_cosine_sim']:.4f}")

del preprocess_model, audio2coeff, animate, exp_net
torch.cuda.empty_cache()